In [ ]:
# ดาวน์โหลดไฟล์ dna.csv จากอินเทอร์เน็ต
import urllib.request
import os

url = 'https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv'
filename = 'bbc_text_cls.csv'

# ตรวจสอบว่าไฟล์มีอยู่แล้วหรือไม่
if not os.path.exists(filename):
    print(f"⏬ กำลังดาวน์โหลด {filename}...")
    urllib.request.urlretrieve(url, filename)
    print(f"✅ ดาวน์โหลด {filename} สำเร็จ!")
else:
    print(f"✅ ไฟล์ {filename} มีอยู่แล้ว (ข้ามการดาวน์โหลด)")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# ตั้งค่า font สำหรับภาษาไทย
import matplotlib.font_manager as fm

font_path = '../../font/Prompt/Prompt-Regular.ttf'
font_prop = fm.FontProperties(fname=font_path)
fm.fontManager.addfont(font_path)
font_name = font_prop.get_name()
plt.rcParams['font.family'] = font_name
plt.rcParams['axes.unicode_minus'] = False

print(f"✅ ตั้งค่า Font ภาษาไทยเป็น '{font_name}' เรียบร้อย")